<a href="https://colab.research.google.com/github/farieu/data-analysis/blob/OutrosModelos/ModelosAdicionais(RandomForest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RandomForest

#### Importação de Bibliotecas e Processamento

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, MultiLabelBinarizer, FunctionTransformer, OneHotEncoder, Binarizer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [2]:
df = pd.read_csv('/content/drive/MyDrive/BackEnd/GoodReads_100k_books.csv')

Etapa da tentativa de realização de um [Pipeline](https://github.com/farieu/data-analysis/tree/Pipeline).

In [3]:
df.dropna(subset=['title', 'desc', 'genre', 'bookformat'], inplace=True)

numeric_features = ['pages', 'totalratings', 'reviews']
categorical_features = ['author', 'bookformat', 'genre', 'title']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])


X = df.drop(columns=['rating', 'desc', 'isbn', 'isbn13', 'img', 'link'])
y = df['rating']

Como foi percebido durante os treinamentos com Regressão Logística, optei por binarizar a partir do 3.5, portanto:


*   Notas abaixo de 3.5, serão binarizadas e atríbuidas a classe 0.
*   Notas igual ou acima de 3.5 serão binarizadas para a classe 1.



In [4]:
binarizer = Binarizer(threshold=3.5)
y_binary = binarizer.fit_transform(y.values.reshape(-1, 1)).ravel()

df_preprocessed = pipeline.fit_transform(X)

df_preprocessed_df = pd.DataFrame(df_preprocessed.toarray() if isinstance(df_preprocessed, np.ndarray) else df_preprocessed)

Separação do conjunto de treino e de teste (80% treino e 20% teste).

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_preprocessed, y_binary, test_size=0.2, random_state=42)

#### Treinamento do Random Forest

Instanciamento do Random Forest, treinamento e avaliação do modelo.

In [6]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Avaliar o desempenho do modelo, relatório de classificação e matriz de confusão.
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred))

Acurácia do modelo: 0.88

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0       0.84      0.04      0.07      2043
         1.0       0.88      1.00      0.94     14768

    accuracy                           0.88     16811
   macro avg       0.86      0.52      0.50     16811
weighted avg       0.88      0.88      0.83     16811


Matriz de Confusão:
 [[   76  1967]
 [   15 14753]]


O resultado do modelo Random Forest, quando comparado ao modelo de [Regressão Linear](https://github.com/farieu/data-analysis/blob/AvaliacaoModeloRL/AvaliacaoModeloRL.ipynb), apresenta resultados interessantes.


---

De forma geral, o modelo apresenta alta acurácia, de 88%, mas as métricas para cada classe mostram um problema significativo na identificação de livros com má avaliação (classe 0).

O recall da classe 0 é MUITO baixo, indicando que o modelo não consegue identificar a maioria de má avaliação, classificando-os erroneamente como "boa avaliação". Esse comportamento é evidênciado pela quantidade alta de **falsos positivos (1967)**, o que mostra um viés do modelo em favor da classe 1.


Entretanto, para a classe de boa avaliação, o modelo consegue identificar corretamente todas as instâncias, sem deixar nenhum livro bem avaliado passar despercebido. Isso também é perceptível pelo F1-Score de 94%.

---

O modelo demonstrou similaridade com o **Treinamento 4** de Regressão Logística, o que sugere um possível desbalanceamento das classes, sendo necessário ajustes futuros no modelo.